# D&D Rules RAG Pipeline with LangChain 1.0

This notebook demonstrates a **Retrieval-Augmented Generation (RAG)** pipeline using LangChain 1.0 to answer questions about D&D rules from our markdown documents.

We will cover:
1. **Environment Setup**: Loading API keys and configuring LangSmith tracing.
2. **Indexing**: Loading documents, splitting them into chunks, and storing embeddings in Qdrant.
3. **RAG Agent**: Building an agent with a retrieval tool for flexible Q&A.
4. **RAG Chain**: A fast 2-step approach with a single LLM call per query.

## 1. Environment Setup

We need to set our OpenAI API Key for embeddings and the LLM. We also enable LangSmith tracing for observability.

In [1]:
import os
import getpass

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")

_set_if_undefined("OPENAI_API_KEY")

# LangSmith Tracing (optional but recommended)
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "D&D RAG Pipeline"
_set_if_undefined("LANGSMITH_API_KEY")

## 2. Indexing

The indexing phase involves:
1. **Loading** documents from our PDFs folder (markdown files)
2. **Splitting** them into manageable chunks
3. **Storing** embeddings in a Qdrant vector store

### 2.1 Loading Documents

In [2]:
from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Path to our markdown documents
PDFS_DIR = Path("PDFs")

# Load all markdown files
loader = DirectoryLoader(
    str(PDFS_DIR),
    glob="**/*.md",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True,
)

docs = loader.load()
print(f"Loaded {len(docs)} documents")

# Show document info
for doc in docs:
    print(f"  - {Path(doc.metadata['source']).name}: {len(doc.page_content):,} characters")

100%|██████████| 3/3 [00:00<00:00, 1697.87it/s]

Loaded 3 documents
  - UA_Eberron_v1.1.md: 18,921 characters
  - DMBasicRulesv.0.3.md: 250,251 characters
  - PlayerDnDBasicRules.md: 572,204 characters


### 2.2 Splitting Documents

We use `RecursiveCharacterTextSplitter` to break documents into chunks that fit within model context windows and are retrievable individually.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)

all_splits = text_splitter.split_documents(docs)
print(f"Split into {len(all_splits)} chunks")

# Preview a chunk
print(f"\nExample chunk (first 500 chars):\n{all_splits[0].page_content[:500]}")

Split into 1095 chunks

Example chunk (first 500 chars):
Unearthed Arcana: Eberron

Welcome to the first installment of Unearthed Arcana, a monthly workshop where D&D R&D shows off a variety of new and interesting pieces of RPG design for use at your gaming table. You can think of the material presented in this series as similar to the first wave of the fifth edition playtest. These game mechanics are in draft form, usable in your campaign but not fully tempered by playtests and design iterations. They are highly volatile and might be unstable; if you


### 2.3 Storing in Qdrant

We use Qdrant as our vector store with OpenAI embeddings. Qdrant runs in-memory for this demo, but can be configured for persistent storage.

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Initialize embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create in-memory Qdrant client
client = QdrantClient(":memory:")

# Get embedding dimension
vector_size = len(embeddings.embed_query("test"))
print(f"Embedding dimension: {vector_size}")

# Create collection
COLLECTION_NAME = "dnd_rules"

if not client.collection_exists(COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
    )
    print(f"Created collection: {COLLECTION_NAME}")

# Create vector store
vector_store = QdrantVectorStore(
    client=client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
)

# Add documents
print("Adding documents to vector store...")
document_ids = vector_store.add_documents(documents=all_splits)
print(f"Added {len(document_ids)} document chunks to Qdrant")

Embedding dimension: 1536
Created collection: dnd_rules
Adding documents to vector store...
Added 1095 document chunks to Qdrant


### 2.4 Test Retrieval

Let's verify our vector store is working by running a quick similarity search.

In [5]:
# Test query
test_query = "What are the warforged racial traits?"
results = vector_store.similarity_search(test_query, k=2)

print(f"Query: {test_query}\n")
for i, doc in enumerate(results, 1):
    source = Path(doc.metadata.get('source', 'Unknown')).name
    print(f"Result {i} (from {source}):")
    print(f"{doc.page_content[:300]}...\n")

Query: What are the warforged racial traits?

Result 1 (from UA_Eberron_v1.1.md):
Traits
As a warforged, you have the following racial traits.
  Ability Score Increase. Your Strength and Constitution scores increase by 1.
  Size. Warforged are generally broader and heavier than humans. Your size is Medium.
  Speed. Your base walking speed is 30 feet.
  Composite Plating. Your con...

Result 2 (from UA_Eberron_v1.1.md):
Razorclaw
As a razorclaw shifter, you make swift, slashing strikes in battle.
Ability Score Increase. Your Dexterity score increases by 1.
Shifting Feature. While shifting, you can make an unarmed strike as a bonus action. You can use your Dexterity for its attack roll and damage bonus, and this att...



## 3. RAG Agent

The **RAG Agent** approach uses LangChain 1.0's `create_agent` to build an agent that can decide when and how to retrieve information. This is flexible and can handle multi-step queries.

### 3.1 Create Retrieval Tool

In [6]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_dnd_rules(query: str):
    """Retrieve information from D&D rulebooks to help answer questions about rules, races, classes, spells, and game mechanics."""
    retrieved_docs = vector_store.similarity_search(query, k=4)
    serialized = "\n\n".join(
        f"Source: {Path(doc.metadata.get('source', 'Unknown')).name}\nContent: {doc.page_content}"
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

tools = [retrieve_dnd_rules]

### 3.2 Create the Agent

In [7]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

# Initialize the model
model = init_chat_model("gpt-4o-mini")

# System prompt for the agent
system_prompt = """
You are an expert Dungeon Master assistant with access to D&D rulebooks.
Use the retrieve_dnd_rules tool to look up rules, racial traits, class features, 
spells, and other game mechanics when answering questions.

Always cite which rulebook the information comes from.
If you're unsure about something, say so rather than making up rules.
"""

# Create the agent
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
)

print("RAG Agent created!")

RAG Agent created!


### 3.3 Test the Agent

In [8]:
# Test the agent with a D&D question
query = "What are the racial traits of Warforged?"

print(f"Question: {query}\n")
print("=" * 60)

for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Question: What are the racial traits of Warforged?

================================ Human Message =================================

What are the racial traits of Warforged?
================================== Ai Message ==================================
Tool Calls:
  retrieve_dnd_rules (call_411pYFGk6OKuUlZO0yOsU0HV)
 Call ID: call_411pYFGk6OKuUlZO0yOsU0HV
  Args:
    query: Warforged racial traits
================================= Tool Message =================================
Name: retrieve_dnd_rules

Source: UA_Eberron_v1.1.md
Content: Traits
As a warforged, you have the following racial traits.
  Ability Score Increase. Your Strength and Constitution scores increase by 1.
  Size. Warforged are generally broader and heavier than humans. Your size is Medium.
  Speed. Your base walking speed is 30 feet.
  Composite Plating. Your construction incorporates wood and metal, granting you a +1 bonus to Armor Class.
  Living Construct. Even though you were constructed, you are a living cre

In [9]:
# Test with a more complex query requiring multiple lookups
complex_query = """
I want to play a shifter character. What subraces are available, 
and what are their shifting features?
"""

print(f"Question: {complex_query}\n")
print("=" * 60)

for step in agent.stream(
    {"messages": [{"role": "user", "content": complex_query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Question: 
I want to play a shifter character. What subraces are available, 
and what are their shifting features?


================================ Human Message =================================


I want to play a shifter character. What subraces are available, 
and what are their shifting features?

================================== Ai Message ==================================
Tool Calls:
  retrieve_dnd_rules (call_cbXCdfi777z63zrBfBPAREbB)
 Call ID: call_cbXCdfi777z63zrBfBPAREbB
  Args:
    query: shifter subraces and shifting features
================================= Tool Message =================================
Name: retrieve_dnd_rules

Source: UA_Eberron_v1.1.md
Content: While shifting, you gain temporary hit points equal to your level + your Constitution bonus (minimum of 1). You also gain a feature that depends on your shifter subrace, described below.
You must finish a short or long rest before you can shift again.
Languages. You can speak, read, and write Common and Syl

## 4. RAG Chain (2-Step Approach)

The **RAG Chain** is a faster, simpler approach that always retrieves context and uses a single LLM call. This is ideal for straightforward Q&A where you always want to search.

| Approach | LLM Calls | Flexibility | Best For |
|----------|-----------|-------------|----------|
| **RAG Agent** | 2+ | High | Complex, multi-step queries |
| **RAG Chain** | 1 | Low | Fast, simple Q&A |

### 4.1 Create the RAG Chain with Middleware

In [10]:
from typing import Any
from langchain_core.documents import Document
from langchain.agents import AgentState, create_agent
from langchain.agents.middleware import AgentMiddleware

# Extended state to store retrieved documents
class RAGState(AgentState):
    context: list[Document]

# Middleware that retrieves and injects context
class RetrieveContextMiddleware(AgentMiddleware[RAGState]):
    state_schema = RAGState
    
    def before_model(self, state: AgentState) -> dict[str, Any] | None:
        """Retrieve context before the model runs."""
        last_message = state["messages"][-1]
        
        # Get the query text
        query = last_message.content if hasattr(last_message, 'content') else str(last_message)
        
        # Retrieve relevant documents
        retrieved_docs = vector_store.similarity_search(query, k=4)
        
        # Format context
        docs_content = "\n\n---\n\n".join(
            f"**Source: {Path(doc.metadata.get('source', 'Unknown')).name}**\n{doc.page_content}"
            for doc in retrieved_docs
        )
        
        # Create augmented message with context
        augmented_content = (
            f"{query}\n\n"
            f"---\n"
            f"Use the following context from D&D rulebooks to answer:\n\n"
            f"{docs_content}"
        )
        
        # Return updated state
        return {
            "messages": [last_message.model_copy(update={"content": augmented_content})],
            "context": retrieved_docs,
        }

# Create the RAG chain (agent with no tools, just middleware)
rag_chain = create_agent(
    model=model,
    tools=[],  # No tools - context is injected via middleware
    middleware=[RetrieveContextMiddleware()],
    system_prompt="""
You are an expert Dungeon Master assistant. Answer questions about D&D rules 
based on the context provided. Always cite which rulebook the information comes from.
If the context doesn't contain the answer, say so.
""",
)

print("RAG Chain created!")

RAG Chain created!


### 4.2 Test the RAG Chain

In [11]:
# Test the RAG chain
query = "What is the Artificer wizard tradition?"

print(f"Question: {query}\n")
print("=" * 60)

for step in rag_chain.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Question: What is the Artificer wizard tradition?

================================ Human Message =================================

What is the Artificer wizard tradition?
================================ Human Message =================================

What is the Artificer wizard tradition?

---
Use the following context from D&D rulebooks to answer:

**Source: UA_Eberron_v1.1.md**
New Wizard Tradition: Artificer
Artificers are a key part of the world of Eberron. They illustrate the evolution of magic from a wild, unpredictable force to one that is becoming available to the masses. Magic items are part of everyday life in the Five Nations of Khorvaire; with an artificer in your party, they become part of every adventuring expedition.
  The artificer was a separate class in prior editions of the Eberron setting, a melee combatant who specialized in mystically enhanced arms and armor. The fifth edition rules treat the artificer as a new wizard tradition that focuses on mystical invent

In [12]:
# Another test
query = "How do dragonmarks work in Eberron?"

print(f"Question: {query}\n")
print("=" * 60)

for step in rag_chain.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Question: How do dragonmarks work in Eberron?

================================ Human Message =================================

How do dragonmarks work in Eberron?
================================ Human Message =================================

How do dragonmarks work in Eberron?

---
Use the following context from D&D rulebooks to answer:

**Source: UA_Eberron_v1.1.md**
In addition, whenever you fail a death saving throw, you can spend an action point to make it a success.

Dragonmarks
Dragonmarks are elaborate skin patterns, similar to tattoos, that grant their bearers innate spellcasting abilities. Each type of mark is tied to large, extended families that each control a different industry or trade in Eberron. Not every member of a given family possesses a dragonmark; conversely, merely possessing a dragonmark does not grant special status within the house.

You must use a feat to gain a dragonmark. You are a member of its corresponding dragonmarked house (or houses, in the case o

## 5. Interactive Demo

Use this cell to ask your own questions about D&D rules!

In [ ]:
def ask_dnd_question(question: str, use_agent: bool = True):
    """
    Ask a question about D&D rules.
    
    Args:
        question: Your D&D rules question
        use_agent: True for RAG Agent (flexible), False for RAG Chain (fast)
    """
    print(f"🎲 Question: {question}")
    print(f"📚 Using: {'RAG Agent' if use_agent else 'RAG Chain'}")
    print("=" * 60 + "\n")
    
    pipeline = agent if use_agent else rag_chain
    
    response = pipeline.invoke(
        {"messages": [{"role": "user", "content": question}]}
    )
    
    print(response["messages"][-1].content)

# Example usage:
ask_dnd_question("What ability score increases do changelings get?")

In [ ]:
# Try your own question!
ask_dnd_question("How does the shifting ability work for shifters?", use_agent=False)

## Summary

In this notebook, we built a complete RAG pipeline for D&D rules using LangChain 1.0:

1. **Indexed** 3 D&D rulebook documents (markdown format) into Qdrant vector store
2. **Created a RAG Agent** with a retrieval tool for flexible, multi-step queries
3. **Created a RAG Chain** with middleware for fast, single-call Q&A

### Key Takeaways

| Component | Purpose |
|-----------|--------|
| `DirectoryLoader` | Load markdown files from disk |
| `RecursiveCharacterTextSplitter` | Split documents into retrievable chunks |
| `QdrantVectorStore` | Store and search embeddings |
| `@tool` decorator | Create retrieval tool for agent |
| `create_agent` | Build LangChain 1.0 agent |
| `AgentMiddleware` | Inject context for RAG chain |

### Next Steps

- Add conversational memory for multi-turn interactions
- Deploy with LangServe for API access
- Add more rulebooks to the knowledge base
- Implement query rewriting for better retrieval